# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Use LabelEncoder to numerify object type columns

le = LabelEncoder()

for col in df.select_dtypes(['object']):
    le.fit(df[col])
    df[col] = le.transform(df[col])

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,2,1,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,1,105000.0,2,2,1,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,1,56000.0,2,2,1,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,3,92000.0,2,2,1,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,1,52000.0,0,2,1,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


In [5]:
df.loan_status.describe()

count    68817.000000
mean         0.994958
std          0.070831
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: loan_status, dtype: float64

## Split the Data into Training and Testing

In [6]:
from sklearn.model_selection import train_test_split
y = df.loan_status.to_frame()
X = df.drop(columns='loan_status')



In [7]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.805542,0.0,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.714932,0.0,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.0,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.000000,0.0,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,1.000000,0.0,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,1.000000,0.0,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,2.000000,0.0,999.000000,18.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [8]:
# Check the balance of our target values
y['loan_status'].value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [9]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
39692,10000.0,0.1131,328.86,1,49000.0,1,1,0,24.54,0.0,...,100.0,0.0,1.0,0.0,309140.0,56262.0,9500.0,54197.0,0,0
37830,21000.0,0.1072,684.74,3,57400.0,0,1,0,34.52,0.0,...,100.0,0.0,0.0,0.0,98100.0,35707.0,64700.0,30000.0,0,0
64531,35000.0,0.0881,1109.90,2,180000.0,0,1,0,3.01,0.0,...,94.7,0.0,0.0,0.0,327877.0,1222.0,41500.0,26177.0,0,0
28730,20000.0,0.0819,628.49,2,80000.0,1,0,0,26.51,0.0,...,90.0,0.0,0.0,0.0,143541.0,81162.0,14500.0,111641.0,0,0
35102,12600.0,0.1356,427.96,1,71000.0,1,0,0,22.38,1.0,...,88.9,0.0,0.0,0.0,316682.0,33433.0,26500.0,50182.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18872,35000.0,0.0881,1109.90,1,400000.0,1,0,0,13.79,0.0,...,97.6,0.0,1.0,0.0,631699.0,189478.0,28600.0,244399.0,0,0
6415,16800.0,0.0881,532.76,2,31000.0,2,2,0,20.48,0.0,...,100.0,0.0,0.0,0.0,20400.0,11771.0,3500.0,16900.0,0,0
12632,25000.0,0.1102,818.71,1,90000.0,0,0,0,15.27,1.0,...,96.6,12.5,0.0,0.0,195681.0,40625.0,40600.0,33181.0,0,0
24022,14000.0,0.1072,456.49,1,180000.0,0,0,0,12.35,2.0,...,73.9,50.0,0.0,0.0,754884.0,49607.0,11000.0,63655.0,0,0


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [10]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [11]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [12]:
# Scale the training and testing data
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled = X_scaler.transform(X_train)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [13]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [14]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7633263634568975

In [15]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   57,    30],
       [ 2200, 14918]], dtype=int64)

In [16]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.03      0.66      0.87      0.05      0.76      0.56        87
          1       1.00      0.87      0.66      0.93      0.76      0.58     17118

avg / total       0.99      0.87      0.66      0.93      0.76      0.58     17205



In [17]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_

importances_sorted = sorted(zip(brf.feature_importances_, X.columns), reverse=True)
importances_sorted[:20]


[(0.0813794468437654, 'total_rec_prncp'),
 (0.06418489100033847, 'last_pymnt_amnt'),
 (0.06295948018853922, 'total_pymnt_inv'),
 (0.057137400997254374, 'total_pymnt'),
 (0.04766027833531938, 'total_rec_int'),
 (0.02670271626050664, 'int_rate'),
 (0.02349411021992901, 'issue_d'),
 (0.018585321545996108, 'annual_inc'),
 (0.01833373761851926, 'installment'),
 (0.017292525336509743, 'mths_since_recent_inq'),
 (0.01723744299838225, 'dti'),
 (0.01641175995495965, 'tot_hi_cred_lim'),
 (0.016258989852809942, 'all_util'),
 (0.01616334482568218, 'total_bal_ex_mort'),
 (0.01606644792960738, 'max_bal_bc'),
 (0.016035681440844853, 'mo_sin_old_rev_tl_op'),
 (0.015936910368223915, 'out_prncp_inv'),
 (0.015873243094750432, 'il_util'),
 (0.015374796839656165, 'bc_util'),
 (0.015233821519991244, 'revol_bal')]

### Easy Ensemble Classifier

In [18]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
ee = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ee.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [20]:
# Calculated the balanced accuracy score
y_pred = ee.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9246971326814686

In [21]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   79,     8],
       [ 1004, 16114]], dtype=int64)

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.07      0.91      0.94      0.14      0.92      0.85        87
          1       1.00      0.94      0.91      0.97      0.92      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.92      0.86     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Easy Ensemble had a better accuracy score (0.92 vs 0.76)

2. Which model had the best recall score?

    The Easy Ensemble had a better recall score (0.94 vs. 0.87)

3. Which model had the best geometric mean score?

    The Easy Ensemble had a better geometric score (0.92 vs. 0.76)

4. What are the top three features?

    The top three features are total_rec_prncp, last_pymnt_amnt, and total_pymnt_inv. 